In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# data_dir = "/content/drive/MyDrive/dataset"

# data_path = Path(data_dir)
# classes = sorted([p.name for p in data_path.iterdir() if p.is_dir()])
# print("Classes:", classes)


Mounted at /content/drive


YOLO classification requires this structure:

dataset --> train || val || test

We currently have only class folders in GDrive.

We will tell YOLO to auto-split our GDrive dataset into train/val/test inside Colab (without touching Drive).

In [ ]:
!mkdir dataset_yolo # Made a new working dataset in Colab

mkdir: cannot create directory ‘dataset_yolo’: File exists


In [ ]:
# import os
# import shutil
# import random
# from pathlib import Path

# # Source dataset (in Google Drive)
# data_dir = "/content/drive/MyDrive/dataset"
# data_path = Path(data_dir)

# # Get classes
# classes = sorted([p.name for p in data_path.iterdir() if p.is_dir()])
# print("Classes:", classes)

# # Destination folder for YOLO-ready dataset (in Google Drive)
# dest_dir = Path('/content/drive/MyDrive/dataset_yolo')

# # Define split ratios
# train_ratio = 0.7
# val_ratio = 0.2

# # Create directory structure
# for split_type in ['train', 'val', 'test']:
#     for class_name in classes:
#         os.makedirs(dest_dir / split_type / class_name, exist_ok=True)

# # Split each class
# for class_name in classes:
#     source_class_path = data_path / class_name
#     images = [f for f in source_class_path.iterdir() if f.is_file() and f.suffix.lower() in ['.jpg', '.jpeg', '.png']]

#     random.shuffle(images)

#     num_images = len(images)
#     num_train = int(num_images * train_ratio)
#     num_val = int(num_images * val_ratio)
#     num_test = num_images - num_train - num_val

#     train_files = images[:num_train]
#     val_files = images[num_train:num_train + num_val]
#     test_files = images[num_train + num_val:]

#     for f in train_files:
#         shutil.copy(f, dest_dir / 'train' / class_name / f.name)
#     for f in val_files:
#         shutil.copy(f, dest_dir / 'val' / class_name / f.name)
#     for f in test_files:
#         shutil.copy(f, dest_dir / 'test' / class_name / f.name)

# print("✔ Dataset splitting complete!")
# print("Saved to:", dest_dir)

Classes: ['cat', 'cow', 'dog', 'lamb', 'zebra']
✔ Dataset splitting complete!
Saved to: /content/drive/MyDrive/dataset_yolo


# **NOW LET'S TRAIN YOLO WITH THE NEW SPLIT DATASET**

In [ ]:
!pip install ultralytics

In [ ]:
from ultralytics import YOLO

model = YOLO("yolo11n-cls.pt")

model.train(
    data="/content/drive/MyDrive/dataset_yolo",
    epochs=20,
    imgsz=256,
    batch=32
)

Ultralytics 8.3.235 🚀 Python-3.12.12 torch-2.9.0+cpu CPU (AMD EPYC 7B13)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=32, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/drive/MyDrive/dataset_yolo, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=20, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=256, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n-cls.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=1

ultralytics.utils.metrics.ClassifyMetrics object with attributes:

confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7a287147e720>
curves: []
curves_results: []
fitness: 0.9900000095367432
keys: ['metrics/accuracy_top1', 'metrics/accuracy_top5']
results_dict: {'metrics/accuracy_top1': 0.9800000190734863, 'metrics/accuracy_top5': 1.0, 'fitness': 0.9900000095367432}
save_dir: PosixPath('/content/runs/classify/train')
speed: {'preprocess': 0.00041130999989036354, 'inference': 4.7465871899999, 'loss': 1.6980002328637056e-05, 'postprocess': 5.8999999055231456e-05}
task: 'classify'
top1: 0.9800000190734863
top5: 1.0

**Uploaded Image in YOLO Model to Classify**

In [ ]:
from ultralytics import YOLO

model = YOLO("/content/runs/classify/train/weights/best.pt")  # trained model


In [ ]:
from google.colab import files

uploaded = files.upload()
image_path = list(uploaded.keys())[0]
print("Uploaded:", image_path)


Saving images.jpeg to images.jpeg
Uploaded: images.jpeg


In [ ]:
results = model(image_path)

# YOLO returns the predicted class index + probabilities
probs = results[0].probs  # raw probabilities

pred_class_idx = probs.top1  # highest probability class index
confidence = probs.top1conf  # confidence score

# class names come from your folder names
class_names = model.names
pred_class_name = class_names[pred_class_idx]

print(f"Predicted class: {pred_class_name}")
print(f"Confidence: {confidence:.2f}")



image 1/1 /content/images.jpeg: 256x256 zebra 1.00, lamb 0.00, cow 0.00, cat 0.00, dog 0.00, 11.0ms
Speed: 1.7ms preprocess, 11.0ms inference, 0.0ms postprocess per image at shape (1, 3, 256, 256)
Predicted class: zebra
Confidence: 1.00
